In [15]:
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertTokenizer

# leverage checkpoints for Bert2Bert model...
# use BERT's cls token as BOS token and sep token as EOS token
encoder = BertGenerationEncoder.from_pretrained("google-bert/bert-large-uncased", bos_token_id=2, eos_token_id=102)
# add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
decoder = BertGenerationDecoder.from_pretrained(
    "google-bert/bert-large-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=2, eos_token_id=102
)
bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)

# create tokenizer...
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-large-uncased")

input_ids = tokenizer(
    "This is a long article to summarize", add_special_tokens=False, return_tensors="pt"
).input_ids
labels = tokenizer("This is a short summary", return_tensors="pt").input_ids

# train...
logits = bert2bert(input_ids=input_ids, decoder_input_ids=labels, labels=labels).logits

for _ in range(logits.shape[1]):
    predicted_token_id = logits[0, _].argmax(axis=-1)
    predicted_token = tokenizer.decode(predicted_token_id)
    print("predicted : ", predicted_token)
#loss.backward()

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of BertGenerationDecoder were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['bert.encoder.layer.10.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.11.crossattention.self.key.bias', 'bert.encoder.layer.3.crossattention.self.query.weight', 'bert.encoder.layer.6.crossattention.output.dense.bias', 'bert.encoder.layer.11.crossattention.self.key.weight', 'bert.encoder.layer.3.crossattention.self.value.bias', 'bert.encoder.layer.5.crossattention.self.value.weight', 'bert.encoder.layer.23.crossattention.self.query.bias', 'bert.encoder.layer.

predicted :  # # n i s
predicted :  # # n i s
predicted :  # # n i s
predicted :  # # n i s
predicted :  # # n i s
predicted :  # # n i s
predicted :  # # n i s


In [16]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

def generate_text(prompt, num_sentences=3, max_length=100):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')

    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=num_sentences,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.0
        )

    generated_text = []
    for i in range(num_sentences):
        text = tokenizer.decode(output[i], skip_special_tokens=True)
        generated_text.append(text)

    return generated_text

if __name__ == "__main__":
    prompt = "The weather today is"
    generated_sentences = generate_text(prompt, num_sentences=3, max_length=50)
    for sentence in generated_sentences:
        print(sentence)


/Users/lohith/miniconda3/envs/torch/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


the weather today is sunny........ the sun and................................
the weather today is pleasant........ pleasant..................................
the weather today is dry........ the soil is heavy....... the.......................
